In [ ]:
# This process is now automated for Elementary school Lunch in English and Spanish.
# Just do Runtime > Run all (Ctrl+F9)
# Then download the file and import it into Google Calendar, etc. Open the file menu on the left with the folder icon

# If the fragile formatting code breaks, I may need to update things.
# Also perhaps for a new calendar month format or for a menu type

In [ ]:
# Install packages and libraries
!pip install pypdf ics
!pip install emoji nltk
!pip install beautifulsoup4 requests

In [ ]:
# prompt: use pydpf to read text from a pdf at a given url, parse the entries and create an ics-formatted text string for a calendar program import
from urllib.request import urlopen
from urllib.parse import urlparse, urlunparse, quote
from io import BytesIO
from pypdf import PdfReader
import ics
from datetime import datetime
import re
import pytz
import emoji
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import requests
from bs4 import BeautifulSoup


def get_all_pdfs(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        # Check if the request was successful
        response.raise_for_status()
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all anchor tags and extract the href attribute
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        pdf_files = list(set([file for file in links if file.endswith('.pdf')]))
        return pdf_files
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []


def url_to_text(pdf_url):
    try:
        remote_file = urlopen(pdf_url).read()
        memory_file = BytesIO(remote_file)
        reader = PdfReader(memory_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    return text


def is_valid_date(date_string, language):
    """
    Checks if the input string is a valid date in the format "Month Day"
    where the month is a word (e.g., January, February) and the day is a number.
    """
    if language == 'es':
        pattern = r"^(Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)\s+(\d{1,2})$"
    else:
        pattern = r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})$"
    match = re.match(pattern, date_string)
    if not match:
        return False
    else:
        return True


def parse_date_string(date_string, language):
    """Parses a date string into year, month, day."""
    if language == 'es':
        pattern = r"^(Enero|Febrero|Marzo|Abril|Mayo|Junio|Julio|Agosto|Septiembre|Octubre|Noviembre|Diciembre)\s+(\d{1,2})$"
    else:
        pattern = r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2})$"
    match = re.match(pattern, date_string)
    if not match:
        return None
    month_name = match.group(1)
    day = int(match.group(2))

    if language == 'es':
        year = datetime.now().year
        # list of month names
        month_names = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
        month = month_names.index(month_name)+1
        return year, month, day
    elif language == 'en':
        year = datetime.now().year
        month_names = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        month = month_names.index(month_name)+1
        return year, month, day
    else:
        return None


def text_to_ics(text, event_title, language):
    # Create WordNetLemmatizer object
    wnl = WordNetLemmatizer()

    calendar = ics.Calendar()
    lines = text.split('\n')
    #print(lines)
    i=0
    while i < len(lines):
        line = lines[i]
        if is_valid_date(line.strip(), language):
            date_parts = parse_date_string(line.strip(), language)
            if date_parts is not None:
                year, month, day = date_parts
                event = ics.Event()
                event.name = event_title
                event.begin = datetime(year, month, day, tzinfo=pytz.utc) # set begin time
                event.make_all_day()
                extra_content = ics.utils.ContentLine(name="TRANSP", value="TRANSPARENT")
                event.extra.append(extra_content)
                # build description until you reach next date
                event.description = ''
                while i + 1 < len(lines) and not is_valid_date(lines[i + 1].strip(), language) and "Prices" not in lines[i + 1] and "Precios" not in lines[i + 1]:
                    next_line = lines[i + 1].strip()
                    next_line_no_colons: str = re.sub(r':', r'', next_line)
                    next_line_no_colons_low = next_line_no_colons.lower()
                    lemmatized_string = ' '.join([wnl.lemmatize(words) for words in next_line_no_colons_low.split()])
                    with_colons: str = re.sub(r'(\w*)', r':\1:', lemmatized_string)
                    if language == 'es':
                        with_emojis: str = emoji.emojize(string=with_colons, language=language)
                    elif language == 'en':
                        with_emojis: str = emoji.emojize(string=with_colons, language='alias')
                    else:
                        return None
                    only_emoji: str = ''.join([c for c in with_emojis if c in emoji.EMOJI_DATA])
                    line = next_line_no_colons + only_emoji
                    #print(line)
                    event.description += line + '\n'
                    i += 1
                calendar.events.add(event)
        else:
            i += 1
    return calendar


def to_file(ics_string, filename):
    # Save to file:
    if ics_string:
        with open(filename, "w", newline='', encoding='utf-8') as f:
            f.write(ics_string)


def wrap(baseUrl, pdf_files, language):
    if 'en' in language:
        relative_url = [file for file in pdf_files if 'ES' in file and 'Lunch' in file and not 'Spanish' in file]
        event_title = 'DPS Lunch Menu'  # All events in the calendar will have this title
        outfile = 'english.ics'
    elif 'es' in language:
        relative_url = [file for file in pdf_files if 'ES' in file and 'Lunch' in file and 'Spanish' in file]
        event_title = 'DPS Almuerzo Menu'  # All events in the calendar will have this title
        outfile = 'espanol.ics'
    else:
        return False
    link = baseUrl + relative_url[0][5:].replace(" ", "%20")
    #print(link)
    text = url_to_text(link)
    #print(text)
    cal = text_to_ics(text, event_title, language)
    ics_string = cal.serialize()
    #print(ics_string)
    to_file(ics_string, outfile)
    return True

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Now we find the menu from the site
url = 'https://www.dpsnc.net/Page/7089'
baseUrl = 'https://www.dpsnc.net'
pdf_files = get_all_pdfs(url)
pdf_files

['../../site/handlers/filedownload.ashx?moduleinstanceid=16101&dataid=116895&FileName=DPS PreK Whitted Breakfast Feb25.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16101&dataid=116900&FileName=DPS K12 ASSP Feb25.pdf',
 'https://www.dpsnc.net/cms/lib/NC01911152/Centricity/Domain/4/DPS Parents Guide to Student Achievement Spanish.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16449&dataid=116911&FileName=DPS K12 ASSP Feb25 - Spanish.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16121&dataid=109934&FileName=DPS Breakfast - BIC - Cycle - Carb List.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16101&dataid=116898&FileName=DPS ES Lunch Feb25.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16101&dataid=116896&FileName=DPS HS Lunch Feb25.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstanceid=16101&dataid=116902&FileName=DPS K12 Breakfast Feb25.pdf',
 '../../site/handlers/filedownload.ashx?moduleinstancei

In [ ]:
# English code here
wrap(baseUrl, pdf_files, language:='en')

True

In [ ]:
# Spanish code here
wrap(baseUrl, pdf_files, language:='es')

True